# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298443119976                   -0.85    5.2
  2   -8.300190503735       -2.76       -1.25    1.1
  3   -8.300450370975       -3.59       -1.89    3.6
  4   -8.300462689956       -4.91       -2.76    1.6
  5   -8.300464380294       -5.77       -3.03    6.5
  6   -8.300464501158       -6.92       -3.19    1.4
  7   -8.300464572465       -7.15       -3.33    6.6
  8   -8.300464609661       -7.43       -3.47    1.0
  9   -8.300464636173       -7.58       -3.67    1.0
 10   -8.300464637939       -8.75       -3.73    2.0
 11   -8.300464642214       -8.37       -3.93    1.1
 12   -8.300464643967       -8.76       -4.44    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67384488867                   -0.70    5.6
  2   -16.67787672195       -2.39       -1.15    1.4
  3   -16.67919514955       -2.88       -1.87    4.0
  4   -16.67926694067       -4.14       -2.69    3.6
  5   -16.67928397753       -4.77       -3.03    5.8
  6   -16.67928619401       -5.65       -3.46    1.9
  7   -16.67928621941       -7.60       -3.96    1.1
  8   -16.67928622405       -8.33       -4.58    1.9


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32480763608                   -0.56    6.4
  2   -33.33237684782       -2.12       -1.00    1.0
  3   -33.33410781057       -2.76       -1.74    3.4
  4   -33.33648589067       -2.62       -2.39    3.1
  5   -33.33693691718       -3.35       -2.70   14.5
  6   -33.33694381639       -5.16       -3.39    1.6
  7   -33.33694388314       -7.18       -3.93    2.9
  8   -33.33694392108       -7.42       -4.48    2.5


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298204915820                   -0.85    5.4
  2   -8.300239630531       -2.69       -1.59    1.5
  3   -8.300433027344       -3.71       -2.44    4.6
  4   -8.300328510628   +   -3.98       -2.19   15.8
  5   -8.300463987177       -3.87       -3.39    5.1
  6   -8.300464551485       -6.25       -3.81    2.4
  7   -8.300464634200       -7.08       -4.27    1.9


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32569252587                   -0.56    7.1
  2   -33.31725926811   +   -2.07       -1.27    1.1
  3   -16.75749695776   +    1.22       -0.46    8.5
  4   -33.31881832888        1.22       -1.87    7.9
  5   -33.03110057124   +   -0.54       -1.30    4.5
  6   -33.15083898024       -0.92       -1.42    4.9
  7   -33.32667294577       -0.75       -1.92    4.2
  8   -33.33583013290       -2.04       -2.42    1.8
  9   -33.33668143169       -3.07       -2.52    2.6
 10   -33.33684131712       -3.80       -2.85    2.0
 11   -33.33694115444       -4.00       -3.45    2.4
 12   -33.33694227537       -5.95       -3.63    3.9
 13   -33.33694342727       -5.94       -3.90    1.9
 14   -33.33694365565       -6.64       -4.10    2.9


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.